In [181]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Subset, Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd

plt.ion()   # interactive mode
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [161]:
def get_target_label_idx(labels, targets, shots=5, test=False):
    """
    Get the indices of labels that are included in targets.
    :param labels: array of labels
    :param targets: list/tuple of target labels
    :return: list with indices of target labels
    """
    final_list = []
    
    for t in targets:
        if test:
            final_list += np.argwhere(np.isin(labels, t)).flatten().tolist()
        else:
            final_list += np.argwhere(np.isin(labels, t)).flatten().tolist()[:shots]
    
    return final_list

In [162]:
from torch.utils.data import Subset
from PIL import Image
from torchvision.datasets import MNIST

import torchvision.transforms as transforms

class MNIST_Dataset(Dataset):

    def __init__(self, root: str, normal_class=0, shots=5):
        super().__init__()

        self.root = root
        self.n_classes = 2  # 0: normal, 1: outlier
        self.normal_classes = tuple([0,1,2,3,4])
        self.outlier_classes = list(range(0, 10))
        self.outlier_classes.remove(normal_class)
        print("classes: ", self.normal_classes)
#         # MNIST preprocessing: GCN (with L1 norm) and min-max feature scaling to [0,1]
        transform = transforms.Compose([transforms.ToTensor()])

#         target_transform = transforms.Lambda(lambda x: int(x in self.normal_classes))

        train_set = MyMNIST(root=self.root, train=True, download=True,
                            transform=transform)
        # Subset train_set to normal class
        train_idx_normal = get_target_label_idx(train_set.train_labels.clone().data.cpu().numpy(), self.normal_classes, shots=shots)
        self.train_set = Subset(train_set, train_idx_normal)

        test_set = MyMNIST(root=self.root, train=False, download=True,
                                transform=transform)
        
        test_idx_normal = get_target_label_idx(test_set.test_labels.clone().data.cpu().numpy(), self.normal_classes, test=True)
        self.test_set = Subset(test_set, test_idx_normal)

        
        

class MyMNIST(MNIST):
    """Torchvision MNIST class with patch of __getitem__ method to also return the index of a data sample."""

    def __init__(self, *args, **kwargs):
        super(MyMNIST, self).__init__(*args, **kwargs)

    def __getitem__(self, index):
        """Override the original method of the MNIST class.
        Args:
            index (int): Index
        Returns:
            triple: (image, target, index) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.numpy(), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index  # only line changed



In [160]:
""" LeNet architecture implementation
"""
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5,5))
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 5)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [164]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            if phase == 'train':
                model.train()  # Set model to training mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for data in train_loader:
                    inputs, labels, idx = data
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(True):
                        outputs = model(inputs)
                        
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)
                        
                        loss.backward()
                        optimizer.step()
                    
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    
                scheduler.step()
                
                epoch_loss = running_loss / train_size
                epoch_acc = running_corrects.double() / train_size

                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Train', epoch_loss, epoch_acc))
                    
            else:
                
                model.eval()   # Set model to evaluate mode
                
                running_loss = 0.0
                running_corrects = 0
                
                for data in test_loader:
                    inputs, labels, idx = data
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    with torch.set_grad_enabled(False):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
#                         print(preds, labels)
                        loss = criterion(outputs, labels)
                
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                
                epoch_loss = running_loss / test_size
                epoch_acc = running_corrects.double() / test_size
                
                print('{} Loss: {:.4f} Acc: {:.4f}'.format('Val', epoch_loss, epoch_acc))
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_acc

In [196]:
shot_list = [5, 10, 15, 20, 25, 30] # no of images per class 
n_epochs = 5
best_net_dict = {}
best_acc_list = []
for shot in shot_list:
    
    mninst_dataset = MNIST_Dataset(root='data/', shots=shot)
    train_loader = DataLoader(mninst_dataset.train_set, batch_size=5, shuffle=True, num_workers=0)
    test_loader = DataLoader(mninst_dataset.test_set, batch_size=200, shuffle=True, num_workers=0)
    train_size =  len(mninst_dataset.train_set)
    test_size = len(mninst_dataset.test_set)
    
    net = LeNet().to(device)
    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    best_net, best_acc = train_model(net, criterion, optimizer_ft, exp_lr_scheduler,
                           num_epochs=n_epochs)
    
    best_acc_list.append(best_acc.item())
    best_net_dict.update({shot : best_net})

classes:  (0, 1, 2, 3, 4)
Epoch 0/19
----------
Train Loss: 1.6130 Acc: 0.2000
Val Loss: 1.6133 Acc: 0.1911

Epoch 1/19
----------
Train Loss: 1.6126 Acc: 0.2000
Val Loss: 1.6132 Acc: 0.1911

Epoch 2/19
----------
Train Loss: 1.6126 Acc: 0.2000
Val Loss: 1.6131 Acc: 0.1911

Epoch 3/19
----------
Train Loss: 1.6123 Acc: 0.2000
Val Loss: 1.6130 Acc: 0.1911

Epoch 4/19
----------
Train Loss: 1.6121 Acc: 0.2000
Val Loss: 1.6129 Acc: 0.1911

Epoch 5/19
----------
Train Loss: 1.6119 Acc: 0.2000
Val Loss: 1.6128 Acc: 0.1911

Epoch 6/19
----------
Train Loss: 1.6117 Acc: 0.2000
Val Loss: 1.6127 Acc: 0.1911

Epoch 7/19
----------
Train Loss: 1.6115 Acc: 0.2000
Val Loss: 1.6125 Acc: 0.1911

Epoch 8/19
----------
Train Loss: 1.6113 Acc: 0.2000
Val Loss: 1.6125 Acc: 0.1911

Epoch 9/19
----------
Train Loss: 1.6113 Acc: 0.2000
Val Loss: 1.6125 Acc: 0.1911

Epoch 10/19
----------
Train Loss: 1.6113 Acc: 0.2000
Val Loss: 1.6125 Acc: 0.1911

Epoch 11/19
----------
Train Loss: 1.6113 Acc: 0.2000
Val Lo

Val Loss: 1.5874 Acc: 0.2687

Epoch 15/19
----------
Train Loss: 1.5869 Acc: 0.2880
Val Loss: 1.5873 Acc: 0.2707

Epoch 16/19
----------
Train Loss: 1.5868 Acc: 0.2880
Val Loss: 1.5873 Acc: 0.2711

Epoch 17/19
----------
Train Loss: 1.5868 Acc: 0.2880
Val Loss: 1.5873 Acc: 0.2720

Epoch 18/19
----------
Train Loss: 1.5867 Acc: 0.2880
Val Loss: 1.5872 Acc: 0.2724

Epoch 19/19
----------
Train Loss: 1.5867 Acc: 0.2880
Val Loss: 1.5872 Acc: 0.2732

Training complete in 0m 7s
Best val Acc: 0.278653
classes:  (0, 1, 2, 3, 4)
Epoch 0/19
----------
Train Loss: 1.6087 Acc: 0.1933
Val Loss: 1.6061 Acc: 0.2415

Epoch 1/19
----------
Train Loss: 1.6066 Acc: 0.2067
Val Loss: 1.6039 Acc: 0.2218

Epoch 2/19
----------
Train Loss: 1.6042 Acc: 0.2067
Val Loss: 1.6016 Acc: 0.2257

Epoch 3/19
----------
Train Loss: 1.6014 Acc: 0.2600
Val Loss: 1.5991 Acc: 0.2989

Epoch 4/19
----------
Train Loss: 1.5983 Acc: 0.3333
Val Loss: 1.5962 Acc: 0.4168

Epoch 5/19
----------
Train Loss: 1.5945 Acc: 0.4400
Val Lo

In [199]:
print("5 way N shots classification accuracy using LeNet")
best_acc_df = pd.DataFrame({'# images per class' : shot_list,
                            'Accuracy' : best_acc_list})
best_acc_df

5 way N shots classification accuracy using LeNet


,# images per class,Accuracy
0,5,0.191088
1,10,0.220860
2,15,0.196536
3,20,0.196536
4,25,0.278653
5,30,0.654018
